# English translation pending

See the `japanese` branch for the original content.


## English translation pending

See the `japanese` branch for the original content.


## English translation pending

See the `japanese` branch for the original content.


In [ ]:
import sys
from pathlib import Path

NOTEBOOK_DIR = Path().resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("PROJECT_ROOT:", PROJECT_ROOT)

In [ ]:
import numpy as np
import pandas as pd




print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("PROJECT_ROOT:", PROJECT_ROOT)

### English translation pending

See the `japanese` branch for the original content.


## English translation pending

See the `japanese` branch for the original content.


In [ ]:
DATA_DIR = PROJECT_ROOT / "data" / "TESS" / "EK_Dra"
print("DATA_DIR:", DATA_DIR)
print("DATA_DIR.exists():", DATA_DIR.exists())

fits_files = []
if not DATA_DIR.exists():
    print("Warning: English translation pending。English translation pending data/ English translation pending。")
else:
    fits_files = sorted(DATA_DIR.glob("*.fits"))
    print("found", len(fits_files), "FITS files")
    print("first 5 files:")
    for path in fits_files[:5]:
        print("  -", path.name)

fits_files[:3]  # Jupyter English translation pending（Path English translation pending）


### English translation pending

See the `japanese` branch for the original content.


## English translation pending

See the `japanese` branch for the original content.


In [ ]:
import astropy.io.fits as fits

if not DATA_DIR.exists() or not fits_files:
    print("DATA_DIR English translation pending FITS English translation pending、English translation pending。")
else:
    example_path = fits_files[0]
    print("example_path:", example_path)

    # with English translation pending: English translation pending
    with fits.open(example_path) as hdul:
        print("HDU list length:", len(hdul))
        # English translation pending、light curve English translation pendingHDU (index 1) English translation pending
        data_hdu = hdul[1]
        print("Columns in HDU[1]:")
        print(data_hdu.columns)

        # TIME English translation pending PDCSAP_FLUX English translation pending
        time_sample = data_hdu.data["TIME"][:5]
        flux_sample = data_hdu.data["PDCSAP_FLUX"][:5]
        print("TIME sample:", time_sample)
        print("PDCSAP_FLUX sample:", flux_sample)


### English translation pending

See the `japanese` branch for the original content.


## English translation pending

See the `japanese` branch for the original content.


In [ ]:
from src.flarepy_EK_Dra import FlareDetector_EK_Dra

if not DATA_DIR.exists() or not fits_files:
    print("EK Dra English translation pending FITS English translation pending、English translation pending。")
else:
    example_file = fits_files[0]
    print("example_file:", example_file)

    # process_data=False English translation pending、English translation pending
    det = FlareDetector_EK_Dra(file=str(example_file), process_data=False)

    print("data_name:", det.data_name)
    print("len(atessBJD):", len(det.atessBJD))
    print("len(amPDCSAPflux):", len(det.amPDCSAPflux))
    print("flux_mean (English translation pending):", det.flux_mean)

    # English translation pending
    print("first 5 atessBJD:", det.atessBJD[:5])
    print("first 5 amPDCSAPflux:", det.amPDCSAPflux[:5])


### English translation pending

See the `japanese` branch for the original content.


In [ ]:
# English translation pending
arr = np.array([1.0, np.nan, 3.0, np.nan, 5.0])
print("arr:", arr)

mask = ~np.isnan(arr)
print("mask:", mask)

clean = arr[mask]
print("clean:", clean)


### English translation pending

See the `japanese` branch for the original content.
